# Import Libraries 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Loading the Datasets 

In [ ]:
df =pd.read_csv('/kaggle/input/itc-nse-24-year-stock-data/ITC-EQ-01-04-2000-to-31-03-2024.csv', parse_dates=['date'])


### Setting the date Column as Index

In [ ]:
df.set_index('date', inplace=True)

# Data Exploring + Statistical Analysis

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.nunique()

In [ ]:
df.isna().sum().sort_values(ascending=False)

# Handling the missing values 

In [ ]:
df["no_of_trades"].fillna(df["no_of_trades"].mean(), inplace = True)
df["52w_h"].fillna(df["52w_h"].mean(), inplace=True)
df["52w_l"].fillna(df["52w_l"].mean(), inplace=True)


# Plot the stock prices over time

In [ ]:
plt.figure(figsize=(16, 8))

# Plot open, high, low, close, prev_close
df[['open', 'high', 'low', 'close', 'prev._close']].plot(figsize=(15, 8), linewidth=1.5)
plt.title('Stock Price Trends (2000-2024)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Price')
plt.grid(True)
plt.show()

# Calculate moving averages

In [ ]:
df['50_MA'] = df['close'].rolling(window=50).mean()
df['200_MA'] = df['close'].rolling(window=200).mean()

# Plot closing prices with moving averages

In [ ]:

plt.figure(figsize=(12, 8))
plt.plot(df['close'], label='close Price', alpha=0.5)
plt.plot(df['50_MA'], label='50-Day Moving Average', color='orange', linestyle='--')
plt.plot(df['200_MA'], label='200-Day Moving Average', color='red', linestyle='--')

plt.title('Stock Price with Moving Averages', fontsize=16)
plt.xlabel('date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

# Calculate daily price range

In [ ]:

df['Daily Range'] = df['high'] - df['low']

# Plot the daily range (volatility)
plt.figure(figsize=(12, 6))
plt.plot(df['Daily Range'], label='Daily Price Range', color='purple', alpha=0.7)

plt.title('Daily Price Range (Volatility)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Price Range')
plt.grid(True)
plt.show()

# We shall filter the Data for last 2 years. 

In [ ]:
latest_2years_df = df[df.index >= '2022-03-31']
latest_2years_df = latest_2years_df.rename(columns={'prev._close': 'prev_close'})
latest_2years_df.head()


In [ ]:
import plotly.express as px

# Plot interactive line chart for stock prices
fig = px.line(latest_2years_df, x=latest_2years_df.index, y=['open', 'high', 'low', 'close', 'prev_close'],
              title='Stock Price Trends (Last 2 Years)')
fig.update_layout(xaxis_title='date', yaxis_title='Price', template='plotly_dark')
fig.show()

# Candlestick Chart


In [ ]:
import plotly.graph_objects as go

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(x=latest_2years_df.index,
                                     open=latest_2years_df['open'],
                                     high=latest_2years_df['high'],
                                     low=latest_2years_df['low'],
                                     close=latest_2years_df['close'])])

fig.update_layout(title='Candlestick Chart (Last 2 Years)',
                  xaxis_title='date',
                  yaxis_title='Price',
                  template='plotly_dark')
fig.show()

# volume with Color-Coded Price Changes

In [ ]:
# Add a column to indicate if the stock closed higher or lower
# also using .loc[] to avoid SettingWithCopyWarning
latest_2years_df.loc[:, 'Price Change'] = latest_2years_df['close'] - latest_2years_df['open']
latest_2years_df.loc[:, 'Color'] = np.where(latest_2years_df['Price Change'] >= 0, 'green', 'red')


# Plot volume with color-coded price changes
plt.figure(figsize=(12, 6))
plt.bar(latest_2years_df.index, latest_2years_df['volume'], color=latest_2years_df['Color'], alpha=0.7)
plt.title('Trading volume with Price Change (Last 2 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('volume')
plt.grid(True)
plt.show()

# Heatmap of Correlations

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 6))
corr_matrix = latest_2years_df[['open', 'high', 'low', 'close', 'prev_close', 'volume']].corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap (Last 2 Years)', fontsize=16)
plt.show()

# Price Returns Analysis

In [ ]:
# Calculate daily returns
latest_2years_df['Daily Returns'] = latest_2years_df['close'].pct_change() * 100  # Percent change

# Drop NaN values created due to pct_change
latest_2years_df.dropna(subset=['Daily Returns'], inplace=True)

# Quick check of daily returns
latest_2years_df[['close', 'Daily Returns']].head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(latest_2years_df['Daily Returns'], bins=100, kde=True, color='blue')

plt.title('Distribution of Daily Returns (Last 2 Years)', fontsize=16)
plt.xlabel('Daily Returns (%)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
latest_2years_df['Cumulative Returns'] = (1 + latest_2years_df['Daily Returns'] / 100).cumprod()

# Plot cumulative returns
plt.figure(figsize=(12, 6))
plt.plot(latest_2years_df.index, latest_2years_df['Cumulative Returns'], color='green')

plt.title('Cumulative Returns (Last 2 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Cumulative Returns')
plt.grid(True)
plt.show()

# Volatility Analysis

In [ ]:
# Calculate 30-day rolling volatility (standard deviation of returns)
latest_2years_df['30_Day_Volatility'] = latest_2years_df['Daily Returns'].rolling(window=30).std()

# Plot the rolling volatility
plt.figure(figsize=(12, 6))
plt.plot(latest_2years_df.index, latest_2years_df['30_Day_Volatility'], color='red', label='30-Day Rolling Volatility')

plt.title('30-Day Rolling Volatility (Last 2 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Volatility')
plt.legend()
plt.grid(True)
plt.show()

#### Install arch library for GARCH modeling

In [ ]:

!pip install arch

# Volatility Clustering 

In [ ]:
from arch import arch_model

# Fit a GARCH model
model = arch_model(latest_2years_df['Daily Returns'], vol='Garch', p=1, q=1)
garch_fit = model.fit(disp='off')

# Print model summary
print(garch_fit.summary())

# Plot the conditional volatility (predicted by the GARCH model)
plt.figure(figsize=(10, 6))
plt.plot(garch_fit.conditional_volatility, color='purple')
plt.title('Conditional Volatility from GARCH Model (Last 4 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Volatility')
plt.grid(True)
plt.show()

# Technical Analysis

## Simple Moving Average (SMA)

In [ ]:
# 50-day and 200-day Simple Moving Averages
latest_2years_df['SMA_50'] = latest_2years_df['close'].rolling(window=50).mean()
latest_2years_df['SMA_200'] = latest_2years_df['close'].rolling(window=200).mean()

# Plot the closing price with the SMAs
plt.figure(figsize=(12, 6))
plt.plot(latest_2years_df['close'], label='close Price', color='blue')
plt.plot(latest_2years_df['SMA_50'], label='50-Day SMA', color='green')
plt.plot(latest_2years_df['SMA_200'], label='200-Day SMA', color='red')

plt.title('Stock Price with 50-Day and 200-Day SMAs (Last 2 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


##  Exponential Moving Average (EMA)

In [ ]:
# 50-day and 200-day Exponential Moving Averages
latest_2years_df['EMA_50'] = latest_2years_df['close'].ewm(span=50, adjust=False).mean()
latest_2years_df['EMA_200'] = latest_2years_df['close'].ewm(span=200, adjust=False).mean()

# Plot the closing price with the EMAs
plt.figure(figsize=(12, 6))
plt.plot(latest_2years_df['close'], label='close Price', color='blue')
plt.plot(latest_2years_df['EMA_50'], label='50-Day EMA', color='orange')
plt.plot(latest_2years_df['EMA_200'], label='200-Day EMA', color='purple')

plt.title('Stock Price with 50-Day and 200-Day EMAs (Last 2 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

## Bollinger Bands

In [ ]:
# Calculate 20-day moving average and standard deviation
latest_2years_df['SMA_20'] = latest_2years_df['close'].rolling(window=20).mean()
latest_2years_df['STD_20'] = latest_2years_df['close'].rolling(window=20).std()

# Calculate Bollinger Bands
latest_2years_df['Upper Band'] = latest_2years_df['SMA_20'] + (latest_2years_df['STD_20'] * 2)
latest_2years_df['lower Band'] = latest_2years_df['SMA_20'] - (latest_2years_df['STD_20'] * 2)

# Plot the closing price with Bollinger Bands
plt.figure(figsize=(12, 6))
plt.plot(latest_2years_df['close'], label='close Price', color='blue')
plt.plot(latest_2years_df['SMA_20'], label='20-Day SMA', color='orange')
plt.plot(latest_2years_df['Upper Band'], label='Upper Band', color='green')
plt.plot(latest_2years_df['lower Band'], label='lower Band', color='red')

plt.fill_between(latest_2years_df.index, latest_2years_df['Upper Band'], latest_2years_df['lower Band'], color='gray', alpha=0.3)

plt.title('Bollinger Bands (Last 4 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

## Relative Strength Index (RSI)

In [ ]:
# Function to calculate RSI
def calculate_RSI(data, window=14):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

# Calculate 14-day RSI
latest_2years_df['RSI_14'] = calculate_RSI(latest_2years_df['close'])

# Plot RSI
plt.figure(figsize=(12, 6))
plt.plot(latest_2years_df.index, latest_2years_df['RSI_14'], label='14-Day RSI', color='purple')

plt.axhline(y=70, color='red', linestyle='--', label='Overbought (70)')
plt.axhline(y=30, color='green', linestyle='--', label='Oversold (30)')

plt.title('14-Day RSI (Last 2 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('RSI')
plt.legend()
plt.grid(True)
plt.show()


# Generating Buy and Sell signals using Indicators 


In [ ]:
## Defining Buy and Sell Conditions

# Buy Signal
latest_2years_df['Buy_Signal'] = (
    (latest_2years_df['SMA_50'] > latest_2years_df['SMA_200']) &  # Golden Cross
    (latest_2years_df['RSI_14'] < 30) &  # RSI Oversold
    (latest_2years_df['close'] < latest_2years_df['lower Band'])  # Below lower Bollinger Band
)

# Sell Signal
latest_2years_df['Sell_Signal'] = (
    (latest_2years_df['SMA_50'] < latest_2years_df['SMA_200']) &  # Death Cross
    (latest_2years_df['RSI_14'] > 70) &  # RSI Overbought
    (latest_2years_df['close'] > latest_2years_df['Upper Band'])  # Above Upper Bollinger Band
)

In [ ]:
## Plot Buy and Sell Signals on the Stock Chart

plt.figure(figsize=(14, 8))

# Plot the closing price
plt.plot(latest_2years_df['close'], label='close Price', color='blue')

# Plot Buy Signals (Green Markers)
plt.plot(latest_2years_df[latest_2years_df['Buy_Signal']].index, 
         latest_2years_df['close'][latest_2years_df['Buy_Signal']], 
         marker='^', color='green', markersize=10, label='Buy Signal', linestyle='None')

# Plot Sell Signals (Red Markers)
plt.plot(latest_2years_df[latest_2years_df['Sell_Signal']].index, 
         latest_2years_df['close'][latest_2years_df['Sell_Signal']], 
         marker='v', color='red', markersize=10, label='Sell Signal', linestyle='None')

# Add labels and title
plt.title('Stock Buy/Sell Signals (Last 2 Years)', fontsize=16)
plt.xlabel('date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
## To summarize when the signals were generated, printing the dates of the buy and sell signals.

# Print Buy Signals
buy_dates = latest_2years_df[latest_2years_df['Buy_Signal']].index
print("Buy Signals Generated on:")
print(buy_dates)

# Print Sell Signals
sell_dates = latest_2years_df[latest_2years_df['Sell_Signal']].index
print("\nSell Signals Generated on:")
print(sell_dates)

# Re-checking the Statergy 

In [ ]:
# Initialize variables for backtesting
initial_capital = 50000  # Starting with $50,000
position = 0  # Whether we hold a position (0 means no position, 1 means holding stock)
cash = initial_capital  # Starting cash
portfolio_value = []  # Track portfolio value over time

# Ensure portfolio value starts on the first day
for i in range(len(latest_2years_df)):
    if latest_2years_df['Buy_Signal'].iloc[i] and position == 0:
        # Buy stock
        position = cash / latest_2years_df['close'].iloc[i]
        cash = 0
        print(f"Buy on {latest_2years_df.index[i]} at {latest_2years_df['close'].iloc[i]:.2f}")
    
    elif latest_2years_df['Sell_Signal'].iloc[i] and position > 0:
        # Sell stock
        cash = position * latest_2years_df['close'].iloc[i]
        position = 0
        print(f"Sell on {latest_2years_df.index[i]} at {latest_2years_df['close'].iloc[i]:.2f}")
    
    # Calculate portfolio value at each step
    if position > 0:
        portfolio_value.append(position * latest_2years_df['close'].iloc[i])
    else:
        portfolio_value.append(cash)

# Ensure the length of portfolio_value matches the DataFrame index
if len(portfolio_value) != len(latest_2years_df):
    print(f"Length mismatch: {len(portfolio_value)} values, {len(latest_2years_df)} rows")

# Add the portfolio value to the dataframe for visualization
latest_2years_df['Portfolio_Value'] = portfolio_value

# Plot portfolio value over time
plt.figure(figsize=(14, 8))
plt.plot(latest_2years_df['Portfolio_Value'], label='Portfolio Value', color='orange')
plt.title('Portfolio Value Over Time', fontsize=16)
plt.xlabel('date')
plt.ylabel('Portfolio Value (INR)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Calculate portfolio returns as the percentage change in Portfolio Value
latest_2years_df['Portfolio_Returns'] = latest_2years_df['Portfolio_Value'].pct_change()

# Drop NaN values that may appear after calculating percentage change
latest_2years_df.dropna(subset=['Portfolio_Returns'], inplace=True)

# Inspect the first few rows
print(latest_2years_df[['Portfolio_Value', 'Portfolio_Returns']].head())

In [ ]:
# Ensure 'date' is a column and not an index
if 'date' not in latest_2years_df.columns:
    latest_2years_df = latest_2years_df.reset_index()

# Calculate the 200-day Moving Average (200 MA)
latest_2years_df.loc[:, '200_MA'] = latest_2years_df['prev_close'].rolling(window=200).mean()

# Calculate Bollinger Bands (20-day moving average with standard deviation)
latest_2years_df.loc[:, '20_MA'] = latest_2years_df['prev_close'].rolling(window=20).mean()
latest_2years_df.loc[:, 'BB_upper'] = latest_2years_df['20_MA'] + (latest_2years_df['prev_close'].rolling(window=20).std() * 2)
latest_2years_df.loc[:, 'BB_lower'] = latest_2years_df['20_MA'] - (latest_2years_df['prev_close'].rolling(window=20).std() * 2)

# Calculate the 50-day Moving Average (50 MA)
latest_2years_df.loc[:, '50_MA'] = latest_2years_df['prev_close'].rolling(window=50).mean()

# Plot with a dark theme
plt.style.use('dark_background')
plt.figure(figsize=(16, 10))

# Plot closing price and moving averages
plt.plot(latest_2years_df['date'], latest_2years_df['prev_close'], label='prev_close Price', color='cyan', linewidth=1.2)
plt.plot(latest_2years_df['date'], latest_2years_df['20_MA'], label='20 MA', color='yellow', linewidth=1)
plt.plot(latest_2years_df['date'], latest_2years_df['50_MA'], label='50 MA', color='orange', linewidth=1)
plt.plot(latest_2years_df['date'], latest_2years_df['200_MA'], label='200 MA', color='green', linewidth=1)

# Plot Bollinger Bands
plt.fill_between(latest_2years_df['date'], latest_2years_df['BB_upper'], latest_2years_df['BB_lower'], color='gray', alpha=0.3, label='Bollinger Bands')

# Format the plot
plt.title('Stock Price with Moving Averages & Bollinger Bands', fontsize=16)
plt.xlabel('date', fontsize=12)
plt.ylabel('Price (INR)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(loc='upper left')

# Display the plot
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a figure with 2 rows for Candlestick and volume
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    row_heights=[0.7, 0.3], vertical_spacing=0.1)

# Candlestick plot
fig.add_trace(go.Candlestick(x=latest_2years_df.index,
                             open=latest_2years_df['open'],
                             high=latest_2years_df['high'],
                             low=latest_2years_df['low'],
                             close=latest_2years_df['close'],
                             name='Candlestick'),
              row=1, col=1)

# volume plot
fig.add_trace(go.Bar(x=latest_2years_df.index, 
                     y=latest_2years_df['volume'], 
                     name='volume'),
              row=2, col=1)

# Update layout
fig.update_layout(title='Stock Price Movement and volume',
                  xaxis_title='date',
                  yaxis_title='Price (INR)',
                  yaxis2_title='volume',
                  showlegend=False)

fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate the 20-day and 50-day moving averages
latest_2years_df['20_MA'] = latest_2years_df['close'].rolling(window=20).mean()
latest_2years_df['50_MA'] = latest_2years_df['close'].rolling(window=50).mean()

# Create a figure with 2 rows for Candlestick, volume, and Moving Averages
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    row_heights=[0.7, 0.3], vertical_spacing=0.1)

# Candlestick plot
fig.add_trace(go.Candlestick(x=latest_2years_df.index,
                             open=latest_2years_df['open'],
                             high=latest_2years_df['high'],
                             low=latest_2years_df['low'],
                             close=latest_2years_df['close'],
                             name='Candlestick'),
              row=1, col=1)

# Add the 20-day and 50-day moving averages to the plot
fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['20_MA'], 
                         mode='lines', 
                         name='20-Day MA', 
                         line=dict(color='blue', width=1)),
              row=1, col=1)

fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['50_MA'], 
                         mode='lines', 
                         name='50-Day MA', 
                         line=dict(color='red', width=1)),
              row=1, col=1)

# volume plot
fig.add_trace(go.Bar(x=latest_2years_df.index, 
                     y=latest_2years_df['volume'], 
                     name='volume', 
                     marker_color='rgba(0, 100, 255, 0.5)'),
              row=2, col=1)

# Update layout: Increase height and set labels
fig.update_layout(title='Stock Price Movement with 20 & 50-Day MA and volume',
                  height=1000,  # Increase the height of the chart
                  xaxis_title='date',
                  yaxis_title='Price (INR)',
                  yaxis2_title='volume',
                  showlegend=True)

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Ensure that the necessary columns exist
latest_2years_df['20_MA'] = latest_2years_df['close'].rolling(window=20).mean()
latest_2years_df['50_MA'] = latest_2years_df['close'].rolling(window=50).mean()
latest_2years_df['200_MA'] = latest_2years_df['close'].rolling(window=200).mean()

# Calculate Bollinger Bands
latest_2years_df['BB_upper'] = latest_2years_df['20_MA'] + (latest_2years_df['close'].rolling(window=20).std() * 2)
latest_2years_df['BB_lower'] = latest_2years_df['20_MA'] - (latest_2years_df['close'].rolling(window=20).std() * 2)

# Create a figure with 2 rows: one for Candlestick and MA, one for volume
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    row_heights=[0.7, 0.3], vertical_spacing=0.1)

# Candlestick plot
fig.add_trace(go.Candlestick(x=latest_2years_df.index,
                             open=latest_2years_df['open'],
                             high=latest_2years_df['high'],
                             low=latest_2years_df['low'],
                             close=latest_2years_df['close'],
                             name='Candlestick'),
              row=1, col=1)


# Add the 20-day, 50-day, and 200-day moving averages to the plot
fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['20_MA'], 
                         mode='lines', 
                         name='20-Day MA', 
                         line=dict(color='blue', width=1)),
              row=1, col=1)

fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['50_MA'], 
                         mode='lines', 
                         name='50-Day MA', 
                         line=dict(color='red', width=1)),
              row=1, col=1)

fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['200_MA'], 
                         mode='lines', 
                         name='200-Day MA', 
                         line=dict(color='green', width=1)),
              row=1, col=1)

# Add Bollinger Bands (upper and lower)
fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['BB_upper'], 
                         mode='lines', 
                         name='Bollinger Upper Band', 
                         line=dict(color='gray', width=1, dash='dash')),
              row=1, col=1)

fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['BB_lower'], 
                         mode='lines', 
                         name='Bollinger lower Band', 
                         line=dict(color='gray', width=1, dash='dash')),
              row=1, col=1)

# volume plot
fig.add_trace(go.Bar(x=latest_2years_df.index, 
                     y=latest_2years_df['volume'], 
                     name='volume', 
                     marker_color='rgba(0, 100, 255, 0.5)'),
              row=2, col=1)

# Update layout: Increase height, add title, labels, and set dark theme
fig.update_layout(title='Stock Price Movement with Moving Averages, Bollinger Bands, and volume',
                  height=1000,  # Height of the chart
                  xaxis_title='date',
                  yaxis_title='Price (INR)',
                  yaxis2_title='volume',
                  showlegend=True,
                  template='plotly_dark')  # Dark theme

# Display the plot
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Ensure that the necessary columns exist
latest_2years_df['20_MA'] = latest_2years_df['close'].rolling(window=20).mean()
latest_2years_df['50_MA'] = latest_2years_df['close'].rolling(window=50).mean()
latest_2years_df['200_MA'] = latest_2years_df['close'].rolling(window=200).mean()

# Calculate Bollinger Bands
latest_2years_df['BB_upper'] = latest_2years_df['20_MA'] + (latest_2years_df['close'].rolling(window=20).std() * 2)
latest_2years_df['BB_lower'] = latest_2years_df['20_MA'] - (latest_2years_df['close'].rolling(window=20).std() * 2)

# Create a figure with 2 rows: one for Candlestick and MA, one for volume
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    row_heights=[0.7, 0.3], vertical_spacing=0.1)

# Candlestick plot
fig.add_trace(go.Candlestick(x=latest_2years_df.index,
                             open=latest_2years_df['open'],
                             high=latest_2years_df['high'],
                             low=latest_2years_df['low'],
                             close=latest_2years_df
                             ['close'],
                             name='Candlestick'),
              row=1, col=1)

# Add the 20-day, 50-day, and 200-day moving averages to the plot
fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['20_MA'], 
                         mode='lines', 
                         name='20-Day MA', 
                         line=dict(color='blue', width=1)),
              row=1, col=1)

fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['50_MA'], 
                         mode='lines', 
                         name='50-Day MA', 
                         line=dict(color='red', width=1)),
              row=1, col=1)

#  reference="top", showticksuffix="last"

fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['200_MA'], 
                         mode='lines', 
                         name='200-Day MA', 
                         line=dict(color='green', width=1)),
              row=1, col=1)

# Add Bollinger Bands (upper and lower)
fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['BB_upper'], 
                         mode='lines', 
                         name='Bollinger Upper Band', 
                         line=dict(color='gray', width=1, dash='dash')),
              row=1, col=1)

fig.add_trace(go.Scatter(x=latest_2years_df.index, 
                         y=latest_2years_df['BB_lower'], 
                         mode='lines', 
                         name='Bollinger lower Band', 
                         line=dict(color='gray', width=1, dash='dash')),
              row=1, col=1)

# volume plot
fig.add_trace(go.Bar(x=latest_2years_df.index, 
                     y=latest_2years_df['volume'], 
                     name='volume', 
                     marker_color='rgba(0, 100, 255, 0.5)'),
              row=2, col=1)

# Update layout: Increase height, add title, labels, and set dark theme
fig.update_layout(title='Stock Price Movement with Moving Averages, Bollinger Bands, and volume',
                  height=1000,  # Height of the chart
                  xaxis_title='date',
                  yaxis_title='Price (INR)',
                  yaxis2_title='volume',
                  showlegend=True,
                  template='plotly_dark',  # Dark theme
                  
                  # Add interactivity for zooming/panning
                  xaxis_rangeslider_visible=True,  # Disable default range slider
                  
                  # Update date formatting and rotation
                  xaxis=dict(tickangle=-45, title_text="date", showline=True, showgrid=True),
                  
                  # Add custom range selector buttons for zooming
                  xaxis2=dict(
                      rangeselector=dict(
                          buttons=list([
                              dict(count=1, label="1M", step="month", stepmode="backward"),
                              dict(count=6, label="6M", step="month", stepmode="backward"),
                              dict(count=1, label="YTD", step="year", stepmode="todate"),
                              dict(count=1, label="1Y", step="year", stepmode="backward"),
                              dict(step="all")
                          ])
                      ),
                      rangeslider=dict(visible=True),  # Custom range slider visible
                      type="date"  # date axis type
                  ))

# Show the plot
fig.show()